In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
backdoor = Backdoor(VggNet, config='11-layer', channels=1, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.MNIST, ImageEntity)

Files already downloaded and verified
Files already downloaded and verified


### Poison setup

In [3]:
patches = ImagePatch((9, 9), 1, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = ([ExpandTransform(1)], [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
loss = backdoor.train(2, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.99% | Loss: 1.8107 | Samples trained: 12496/50000


### Evaluating Backdoor

In [ ]:
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 73.75% | Trojan 73.59% | Difference -0.16%
Base Accuracy on Poison 3.31% | Attack Success Rate (ASR): 64.93%
Average Tensor Distance: 294.60 | Net Tensor Difference 14729876.07


### Creating a Defense

In [ ]:
defense = Defense(backdoor)

### Detection setup

In [ ]:
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [ ]:
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 73.34% | Original Accuracy 73.75% | Difference -0.41%
Patch size (5, 5) | Synthetic Poison Accuracy 72.73% | Original Accuracy 73.75% | Difference -1.02%
Patch size (7, 7) | Synthetic Poison Accuracy 71.80% | Original Accuracy 73.75% | Difference -1.95%
Patch size (9, 9) | Synthetic Poison Accuracy 69.76% | Original Accuracy 73.75% | Difference -3.99%
Patch size (11, 11) | Synthetic Poison Accuracy 69.61% | Original Accuracy 73.75% | Difference -4.14%
Average Accuracy 71.45% | Average Difference -2.30% | Lowest Score: 69.61% | Likihood of Backdoor: Low


### Detection Trojan

In [ ]:
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 73.20% | Original Accuracy 73.59% | Difference -0.39%
Patch size (5, 5) | Synthetic Poison Accuracy 70.22% | Original Accuracy 73.59% | Difference -3.37%
Patch size (7, 7) | Synthetic Poison Accuracy 59.36% | Original Accuracy 73.59% | Difference -14.23%
Patch size (9, 9) | Synthetic Poison Accuracy 52.60% | Original Accuracy 73.59% | Difference -20.99%
Patch size (11, 11) | Synthetic Poison Accuracy 52.53% | Original Accuracy 73.59% | Difference -21.06%
Average Accuracy 61.58% | Average Difference -12.01% | Lowest Score: 52.53% | Likihood of Backdoor: High


### Blocking Patches setup

In [ ]:
labels = {i : i - 1 if i > 0 else 9 for i in range(10)}
patch = ImagePatch((10, 10), 1, 'random')
n = 5000

### Blocking

In [ ]:
blockedloader = defense.block(poisontest, patch, labels, n)

### Testing after Block Base

In [ ]:
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
23.94% Testing complete | Loss: 1.4263 | Accuracy: 0.6048
47.89% Testing complete | Loss: 1.4199 | Accuracy: 0.5928
71.83% Testing complete | Loss: 1.4017 | Accuracy: 0.5944
95.77% Testing complete | Loss: 1.3820 | Accuracy: 0.5910
Testing complete | Loss: 1.3862 | Accuracy: 59.22%


### Testing after Block Trojan

In [ ]:
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
23.94% Testing complete | Loss: 3.4264 | Accuracy: 0.2243
47.89% Testing complete | Loss: 3.4777 | Accuracy: 0.2114
71.83% Testing complete | Loss: 3.4883 | Accuracy: 0.2047
95.77% Testing complete | Loss: 3.5136 | Accuracy: 0.2077
Testing complete | Loss: 3.5181 | Accuracy: 20.84%


### Defense by retraining

In [ ]:
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 0.7622 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 0.7609 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 0.7504 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 0.7120 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 0.7113
Epoch 2/3 | 24.95% | Loss: 0.7655 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 0.7616 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 0.7488 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 0.7107 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 0.7100
Epoch 3/3 | 24.95% | Loss: 0.7672 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 0.7627 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 0.7494 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 0.7107 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 0.7101
Training complete | Net Average Loss: 0.7105 | Total epochs: 3


### Evaluation after retraining

In [ ]:
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 73.75% | Trojan 73.59% | Defense 72.02%
Accuracy on Posion | Base 3.31% | Defense 41.57% | Trojan ASR 64.93%
Difference from Baseline | Trojan -0.16% | Defense -1.73%
Defense Effectiveness | 23.36% decrease in ASR


### Pruning setup

In [ ]:
defense.reset()
layers = ['layers.28', 'layers.26', 'fc1', 'fc2']
amount = 0.8

### Pruning based on l-Infinity norm of weights from layers

In [ ]:
defense.prune(layers, amount)

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 9.78 GiB total capacity; 7.83 GiB already allocated; 164.94 MiB free; 7.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Evaluation after prune

In [ ]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 73.75% | Trojan 73.59% | Defense 15.52%
Accuracy on Posion | Base 3.31% | Defense 10.13% | Trojan ASR 64.93%
Difference from Baseline | Trojan -0.16% | Defense -58.23%
Defense Effectiveness | 54.80% decrease in ASR


### Retraining to recover

In [ ]:
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 1.6479 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 1.6485 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 1.6476 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 1.6398 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.6397
Epoch 2/3 | 24.95% | Loss: 1.6489 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 1.6482 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 1.6471 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 1.6386 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 1.6384
Epoch 3/3 | 24.95% | Loss: 1.6495 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 1.6494 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 1.6476 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 1.6394 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 1.6393
Training complete | Net Average Loss: 1.6392 | Total epochs: 3


### Evaluation after retraining

In [ ]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 73.75% | Trojan 73.59% | Defense 70.15%
Accuracy on Posion | Base 3.31% | Defense 36.25% | Trojan ASR 64.93%
Difference from Baseline | Trojan -0.16% | Defense -3.60%
Defense Effectiveness | 28.68% decrease in ASR
